# Wettervorhersagen mit Hilfe eines Neuronalen Netzes

Diese Datei ist der "Neuronale Netz"-Teil im Vergleich zwischen Linear Regression, Random Forest und Neuronalen Netzwerken im Bezug darauf, welches Modell besser in der Lage dazu ist Wetterdaten (Temperatur und Niederschlag) mit Hilfe von Datensätzen aus drei Wetterstationen, 3 Tage in die Zukunft vorherzusagen. Die zugrundeliegenden Daten liegen in CSV Format vor.

## Datenstruktur
Jede Zeile in den vorliegenden CSV Dateien repräsentieren (Wetter)daten eines Tags in chronologischer Reihenfolge. Die verfügbaren Spalten umfassen die folgenden Features:
- `DATE`
- `MESS_DATUM`
- `QUALITAETS_NIVEAU`
- `LUFTTEMPERATUR`
- `DAMPFDRUCK`
- `BEDECKUNGSGRAD`
- `LUFTDRUCK_STATIONSHOEHE`
- `REL_FEUCHTE`
- `WINDGESCHWINDIGKEIT`
- `LUFTTEMPERATUR_MAXIMUM`
- `LUFTTEMPERATUR_MINIMUM`
- `LUFTTEMP_AM_ERDB_MINIMUM`
- `WINDSPITZE_MAXIMUM`
- `NIEDERSCHLAGSHOEHE`
- `NIEDERSCHLAGSHOEHE_IND`
- `SONNENSCHEINDAUER`
- `SCHNEEHOEHE`

Es ist dabei wichtig zu beachten, dass fehlende Daten mit dem WErt -999 gekennzeichnet sind. Weiterhin ist zu beachten, dass die Wetterstationen unterschiedliche Daten als Aufzeichnungsbeginn haben.

In [ ]:
# Import der nötigen Bibliotheken
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

# Daten importieren
Nachdem wir die für unser Programm nötigen Bibliotheken importiert haben müssen wir nun die Wetterdaten einlesen. Zusätzlich werden die Leerzeichen am Anfang und Ende der Spaltennamen entfernt.

In [ ]:
# Load Data
arber_data = pd.read_csv(cwd + "/data/Arber.csv")
schorndorf_data = pd.read_csv(cwd + "/data/Schorndorf.csv")
straubing_data = pd.read_csv(cwd + "/data/Straubing.csv")

# Strippen
arber_data.columns = arber_data.columns.str.strip()
schorndorf_data.columns = schorndorf_data.columns.str.strip()
straubing_data.columns = straubing_data.columns.str.strip()

# Data Processing
Um unsere Daten gezielt weiterverarbeiten zu können splitten wir sie pro Wetterstation in Numeric und Non-Numeric Data

In [ ]:
# Split numeric and non numeric columns
arber_numeric = arber_data.select_dtypes(include=["float64", "int64"])
arber_non_numeric = arber_data.select_dtypes(exclude=["float64", "int64"])

schorndorf_numeric = schorndorf_data.select_dtypes(include=["float64", "int64"])
schorndorf_non_numeric = schorndorf_data.select_dtypes(exclude=["float64", "int64"])

straubing_numeric = straubing_data.select_dtypes(include=["float64", "int64"])
straubing_non_numeric = straubing_data.select_dtypes(exclude=["float64", "int64"])

## Handeling missing values
Nachdem wir nun Zugriff auf die numerischen Werte haben können wir uns nun mit den Fehlenden Werten beschäftigen. Da wir einige leere Werte haben füllen wir diese mit dem mean value des columns mit Hilfe eines Imputers.

In [ ]:
# Filling missing values with the mean value only on numeric data
imputer = SimpleImputer(strategy="mean")
arber_numeric_filled = pd.DataFrame(imputer.fit_transform(arber_numeric.replace), columns=arber_numeric.columns)
schorndorf_numeric_filled = pd.DataFrame(imputer.fit_transform(schorndorf_numeric), columns=schorndorf_numeric.columns)
straubing_numeric_filled = pd.DataFrame(imputer.fit_transform(straubing_numeric), columns=straubing_numeric.columns)

## Combining back together numeric and non-numeric data
Nachdem wir die fehlenden Werte mit einem Durchschnittswert gefüllt haben können wir die numerischen und nicht numerischen Daten wieder zusammenführen

In [ ]:
# Combining back together the numeric and non numeric data
arber_data_filled = pd.concat([arber_non_numeric.reset_index(drop=True), arber_numeric_filled.reset_index(drop=True)], axis=1)
schorndorf_data_filled = pd.concat([schorndorf_non_numeric.reset_index(drop=True), schorndorf_numeric_filled.reset_index(drop=True)], axis=1)
straubing_data_filled = pd.concat([straubing_non_numeric.reset_index(drop=True), straubing_numeric_filled.reset_index(drop=True)], axis=1)

## Selecting features
Nach der Rückkombination der Daten suchen wir nun die relevanten Features aus unseren Daten aus. Für das Neuronale Netzwerk haben wir uns für die folgenden Features entschieden:
- `DATE`
- `LUFTTEMPERATUR`
- `DAMPFDRUCK`
- `REL_FEUCHTE`
- `WINDGESCHWINDIGKEIT`
- `LUFTTEMPERATUR_MAXIMUM`
- `LUFTTEMPERATUR_MINIMUM`
- `LUFTTEMP_AM_ERDB_MINIMUM`
- `NIEDERSCHLAGSHOEHE`
- `NIEDERSCHLAGSHOEHE_IND`
- `SCHNEEHOEHE`

In [ ]:
# Selecting relevant features
arber_data_selected = arber_data_filled[["DATE", "LUFTTEMPERATUR",  "REL_FEUCHTE", "LUFTTEMPERATUR_MAXIMUM", "LUFTTEMPERATUR_MINIMUM", "LUFTTEMP_AM_ERDB_MINIMUM", "NIEDERSCHLAGSHOEHE", "DAMPFDRUCK", "NIEDERSCHLAGSHOEHE_IND", "SCHNEEHOEHE"]]
schorndorf_data_selected = schorndorf_data_filled[["DATE", "LUFTTEMPERATUR", "REL_FEUCHTE", "NIEDERSCHLAGSHOEHE", "LUFTTEMPERATUR_MAXIMUM", "LUFTTEMPERATUR_MINIMUM", "LUFTTEMP_AM_ERDB_MINIMUM", "DAMPFDRUCK", "NIEDERSCHLAGSHOEHE_IND", "SCHNEEHOEHE"]]
straubing_data_selected = straubing_data_filled[["DATE", "LUFTTEMPERATUR", "REL_FEUCHTE", "NIEDERSCHLAGSHOEHE", "LUFTTEMPERATUR_MAXIMUM", "LUFTTEMPERATUR_MINIMUM", "LUFTTEMP_AM_ERDB_MINIMUM", "DAMPFDRUCK", "NIEDERSCHLAGSHOEHE_IND", "SCHNEEHOEHE"]]


## Beachten des TimeLags
Da Daten der Wetterstationen Schorndorf und Arber nur mit einem TimeLag von 3 Tagen genutzt werden dürfen müssen die jeweiligen Daten noch entsprechend verschoben werden.

In [ ]:
date_format = "%d.%m.%Y"

# Incorporating Time Lag
# Shift Arber and Schorndorf data by 3 days
arber_data_shifted = arber_data_selected.copy()
arber_data_shifted['DATE'] = pd.to_datetime(arber_data_shifted['DATE'], format=date_format) + pd.DateOffset(days=3)
schorndorf_data_shifted = schorndorf_data_selected.copy()
schorndorf_data_shifted['DATE'] = pd.to_datetime(schorndorf_data_shifted['DATE'], format=date_format) + pd.DateOffset(days=3)

## Merging der Daten
Nun können wir alle Daten zu einem Datensatz zusammenführen.

In [ ]:
# Merging the Data
# Convert 'Date' to datetime for merging
straubing_data_selected.loc[:, 'DATE'] = pd.to_datetime(straubing_data['DATE'], format=date_format)

# Add Prefix to data
arber_data_prefixed = arber_data_shifted.add_prefix("arber_")
schorndorf_data_prefixed = schorndorf_data_shifted.add_prefix("schorndorf_")
straubing_data_prefixed = straubing_data_selected.add_prefix("straubing_")

merged_data = straubing_data_prefixed.merge(arber_data_prefixed, left_on='straubing_DATE', right_on="arber_DATE", how='left')
merged_data = merged_data.merge(schorndorf_data_prefixed, how="left", left_on="straubing_DATE", right_on="schorndorf_DATE")

## Löschen und Füllen von Daten
Um alle NaN Daten aus unserem Datensatz zu haben gehen wir nun über die Daten und löschen NaN Daten zudem ersetzen wir -999 (fehlender Messwert) Werte mit einem Durchschnittswert der jeweiligen Zeile

In [ ]:
merged_data = merged_data[~np.any(np.isnan(merged_data), axis=1)]

column_means = merged_data.replace(-999, np.nan).mean()
# Replace NaN values with the corresponding column mean values
# merged_data.fillna(column_means, inplace=True) 
merged_data.replace(-999, column_means, inplace=True)

## Selecting the data for the Neural Network
Da wir unserem Neuronalen Netz keine "Date" Werte füttern können sondern lediglich numerische müssen wir uns eine neue Variable mit nur diesen Werten bilden

In [ ]:
used_data = merged_data.copy()
used_data.drop(columns=["schorndorf_DATE", "straubing_DATE", "arber_DATE"], axis=1)

## Data Normalisation
Theoretisch haben wir die Möglichkeit nun unsere Daten zu normalisieren. Allerdings haben wir uns dafür entschieden dies nicht zu tun, da es den Realitätsbezug der Daten signifikant stört/zerstört.

In [ ]:
# Normalization (Using Min-Max Scaling)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaled_data = scaler.fit_transform(used_data.select_dtypes(include=['float64', 'int64']))
scaled_data = used_data.copy()

## Building Target Day Columns
Um unserem Modell zu ermöglichen Vorhersagen für drei Tage im Vorraus zu machen müssen wir die relevanten Straubing Daten (entweder Temperatur oder Niederschlag) auf drei neue (verschobene) Spalten aufteilen. Am Ende müssen wir die letzten 3 Spalten dropen, da diese auf Grund der Verschiebung nun NaN Werte enthalten

In [ ]:
# Convert scaled_data to DataFrame
scaled_data = pd.DataFrame(scaled_data, columns=column_names)

scaled_data["Target_Day1"] = scaled_data["straubing_LUFTTEMPERATUR"].shift(-1)
scaled_data["Target_Day2"] = scaled_data["straubing_LUFTTEMPERATUR"].shift(-2)
scaled_data["Target_Day3"] = scaled_data["straubing_LUFTTEMPERATUR"].shift(-3)

# scaled_data["Target_Day1"] = scaled_data["straubing_NIEDERSCHLAGSHOEHE"].shift(-1)
# scaled_data["Target_Day2"] = scaled_data["straubing_NIEDERSCHLAGSHOEHE"].shift(-2)
# scaled_data["Target_Day3"] = scaled_data["straubing_NIEDERSCHLAGSHOEHE"].shift(-3)

scaled_data = scaled_data[:-3]

## Selecting X and y
Nun sind unsere Daten soweit vorbereitet, dass wir unser X und y selecten können. Wichtig zu beachten ist, dass X weder die Target_Day noch die originale LUFTTEMPERATUR / NIEDERSCHLAGSHOEHE enthält, da diese Werte ja die sind, die das Neuronale Netzwerk benennen soll. Genau aus diesem Grund besteht y aus diesen Werten

In [ ]:
X = scaled_data.drop(columns=["straubing_LUFTTEMPERATUR", "Target_Day1", "Target_Day2", "Target_Day3"], axis=1)
# X = scaled_data.drop(columns=["straubing_NIEDERSCHLAGSHOEHE", "Target_Day1", "Target_Day2", "Target_Day3"], axis=1)
y = scaled_data[["Target_Day1", "Target_Day2", "Target_Day3"]]

## Train-Test-Split
Wir haben uns hier für einen 80/20 Train-Test-Split entschieden wir teilen die X und y Werte also jeweils im Verhältnis 80/20 in X_train, X_test und y_train, y_test auf.

In [ ]:
train_size = int(len(scaled_data)*0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Dates Train and Test
Um unsere Graphen im Anschluss labeln zu können brauchen wir die korrespondierenden Date Daten zu unseren Werten. Diese bekommen wir aus der vorher angelegten merged_data Variable. Um konsistent zu den anderen Daten zu sein müssen wir auch hier die letzten 3 Spalten droppen und können anschließen unsere dates_train und dates_test auf dem straubing_DATE column mit Hilfe der train_size bilden.

In [ ]:
merged_data = merged_data[:-3]
dates = merged_data[["straubing_DATE"]]
dates_train, dates_test = dates[:train_size], dates[train_size:]

## Creating the Model
Jetzt sind wir so weit, dass wir das Neuronale Netz erstellen können. Wir haben eine auskommentierte Loss Function, die für die Regendaten genutzt werden kann. Die Funktion bestraft negative Werte (die es beim Niederschlag nicht geben kann) stärker und sollte so dem Neuronalen Netz helfen besser zu fitten. Ansonten sind die Werte etwas durch Trial and Error entstanden. Wir haben zwei Hidden Layers, einmal mit 64 und einmal mit 32 Neuronen. Und eine Output Layer mit 3 Neuronen (für jeden Tag ein Neuron.) Auch die Activation Functions sind durch Trial and Error herausgefunden worden. Anfangs war die Hoffnung da, dass das einsetzen einer sigmoid Aktivierungsfunktion oder einer tanh Aktivierungsfunktion das Ergebnis des NN verbessern würde, dies hat sich allerdings nicht bewahrheitet. Für das NN zur Voraussage der Temperatur benutzen wir den MSE als loss function.

In [ ]:
# Initialize the Neural Network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dense(32, activation='elu'))  # Adjust number of neurons
model.add(Dense(3))

# def modified_mse(y_true, y_pred):
#     penalty_weight = 10.0  # Adjust the penalty weight
#     squared_error = tf.square(y_true - y_pred)
#     penalty = tf.where(y_pred < 0, penalty_weight * squared_error, squared_error)
#     return tf.reduce_mean(penalty)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# model.compile(optimizer='adam', loss=modified_mse)

# Train the model
model.fit(X_train, y_train, epochs=250, batch_size=32)  # Adjust epochs and batch_size as needed

# Predictions
predictions = model.predict(X_test)

## Calculate MSE and R2
Nach den Predictions können wir nun die Qualität des NN berechnen dafür berechnen wir einmal den MSE und den R2 Score mit Hilfe der provideden Functions von Sklearn. Um das allerdings tun zu können müssen wir die y_test Daten noch entsprechend in die Daten für die jeweiligen Tagespredictions spalten

In [ ]:
# Prepare the actual values for comparison
N = len(predictions)
actual_day1 = y_test.iloc[:N, 0]
actual_day2 = y_test.iloc[:N, 1]
actual_day3 = y_test.iloc[:N, 2]


# Calculate Mean squared error and R2 Score
from sklearn.metrics import mean_squared_error, r2_score
mse1 = mean_squared_error(actual_day1, predictions[:, 0])
mse2 = mean_squared_error(actual_day2, predictions[:, 1])
mse3 = mean_squared_error(actual_day3, predictions[:, 2])
r2_1 = r2_score(actual_day1, predictions[:, 0])
r2_2 = r2_score(actual_day2, predictions[:, 1])
r2_3 = r2_score(actual_day3, predictions[:, 2])
print("Mean Squared Error1:", mse1)
print("Mean Squared Error2:", mse2)
print("Mean Squared Error3:", mse3)
print("R2 Score 1:", r2_1)
print("R2 Score 2:", r2_2)
print("R2 Score 3:", r2_3)

## Visualizing
Nun können wir die Daten mit Hilfe von Matplotlib visualisieren. Wir haben uns dafür entschieden alle drei Tagesvorhersagen als jeweils einen Subplot dazustellen um direkt vergleichen zu können wie sich die Qulität der Predictions verändert, je weiter man in die Zukunft versucht zu schauen

In [ ]:
dates_pred_day1 = dates_test.shift(-1)  # Exclude the last 3 dates for the 1st day prediction
dates_pred_day2 = dates_test.shift(-2) # Shift for the 2nd day prediction
dates_pred_day3 = dates_test.shift(-3) # Shift for the 3rd day prediction

plt.figure(figsize=(15, 5))


# Plot for Day 1 Prediction
plt.subplot(1, 3, 1)
plt.plot(dates_test, actual_day1, label='Actual', color='blue')
plt.plot(dates_pred_day1, predictions[:, 0], label='Predicted Day 1', color='red', alpha=0.7)
# plt.title('Day 1 Temperature Prediction')
plt.title('Day 1 Rainfall Prediction')
plt.xlabel('Date')
# plt.ylabel('Temperature')
plt.ylabel('Rainfall')
plt.legend()

# Plot for Day 2 Prediction
plt.subplot(1, 3, 2)
plt.plot(dates_test, actual_day2, label='Actual', color='blue')
plt.plot(dates_pred_day2, predictions[:, 1], label='Predicted Day 2', color='orange', alpha=0.7)
# plt.title('Day 2 Temperature Prediction')
plt.title('Day 2 Rainfall Prediction')
plt.xlabel('Date')
# plt.ylabel('Temperature')
plt.ylabel('Rainfall')
plt.legend()

# Plot for Day 3 Prediction
plt.subplot(1, 3, 3)
plt.plot(dates_test, actual_day3, label='Actual', color='blue')
plt.plot(dates_pred_day3, predictions[:, 2], label='Predicted Day 3', color='green', alpha=0.7)
# plt.title('Day 3 Temperature Prediction')
plt.title('Day 3 Rainfall Prediction')
plt.xlabel('Date')
# plt.ylabel('Temperature')
plt.ylabel('Rainfall')
plt.legend()

plt.tight_layout()
plt.show()